In [67]:
try:
    spark.stop()
except Exception:
    pass

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Iceberg via REST")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.local.type", "rest")
    .config("spark.sql.catalog.local.uri", "http://iceberg-rest:8181")
    .config("spark.sql.catalog.local.warehouse", "s3://lake/warehouse")
    .config("spark.sql.catalog.local.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.local.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.local.s3.path-style-access", "true")
    .config("spark.sql.catalog.local.s3.access-key-id", "admin")
    .config("spark.sql.catalog.local.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.local.s3.region", "us-east-1")
    .getOrCreate()
)

spark

25/11/03 01:16:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [68]:
spark.sql("SHOW NAMESPACES IN local").show(truncate=False)

+---------+
|namespace|
+---------+
|raw      |
+---------+



In [69]:
spark.sql("SHOW TABLES IN local.raw").show(truncate=False)

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|raw      |avito    |false      |
+---------+---------+-----------+



In [96]:
tbl = "local.raw.avito"
spark.sql(f"SELECT COUNT(*) AS rows FROM {tbl}").show()

+----+
|rows|
+----+
| 250|
+----+



In [97]:
import os

# Define the export path relative to the current directory of the Jupyter notebook
export_path = os.path.join(os.getcwd(), "exported_avito_table")

# Export the Iceberg table to Parquet format in the current directory
spark.read.format("iceberg").load("local.raw.avito") \
    .write.format("parquet").save(export_path)

# Confirm the files were saved by checking the directory content
print(f"Table exported to: {export_path}")

25/11/03 01:25:08 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

Table exported to: /opt/work/src/notebooks/exported_avito_table


In [71]:
# Load RAW table
raw_df = spark.table("local.raw.avito")   # or local.raw.sarouty

In [72]:
# Créer un DataFrame "Silver" minimal
silver_df = raw_df.select("id").distinct()

# Afficher quelques lignes
silver_df.show(5, truncate=False)

print("✅ Silver dataset initialized with only 'id' column.")
print("Total IDs:", silver_df.count())

+--------+
|id      |
+--------+
|55924046|
|57028285|
|57027014|
|57066031|
|57153879|
+--------+
only showing top 5 rows

✅ Silver dataset initialized with only 'id' column.
Total IDs: 160


In [73]:
from pyspark.sql import functions as F, types as T

# 1) Définir le schéma du JSON dans "payload"
payload_schema = T.StructType([
    T.StructField("id", T.StringType()),
    T.StructField("url", T.StringType()),
    T.StructField("error", T.StringType()),
    T.StructField("title", T.StringType()),
    T.StructField("price_text", T.StringType()),
    T.StructField("breadcrumbs", T.StringType()),
    T.StructField("category", T.StringType()),
    T.StructField("description", T.StringType()),
    T.StructField("attributes", T.StringType()),  # JSON imbriqué sous forme de string
    T.StructField("equipments", T.StringType()),
    T.StructField("seller_name", T.StringType()),
    T.StructField("seller_type", T.StringType()),
    T.StructField("published_date", T.StringType()),
    T.StructField("image_urls", T.StringType()),
])

# 2) Parser le JSON depuis la colonne string "payload"
parsed = (raw_df
    .select(
        *[c for c in raw_df.columns if c != "payload"],  # ex: garder ingest_ts s'il existe
        F.from_json(F.col("payload"), payload_schema).alias("p")
    )
    .filter(F.col("p").isNotNull())  # ignorer les lignes avec JSON invalide
)

# 3) Parser le JSON imbriqué "attributes" -> Map<String,String>
attrs_map = F.from_json(F.col("p.attributes"), T.MapType(T.StringType(), T.StringType()))

# 4) Nettoyages utiles:
# - price_value (MAD) à partir de "price_text" (ex: "6 000 DH" -> 6000.0)
price_value = F.regexp_replace(F.col("p.price_text"), r"[^0-9]", "").cast("double")

# - image_urls -> array<string> en splittant sur " | " et trim de chaque url
image_urls_arr = F.transform(
    F.split(F.col("p.image_urls"), r"\s*\|\s*"),
    lambda x: F.trim(x)
)

# - equipments -> array<string> en splittant sur ";"
equipments_arr = F.transform(
    F.split(F.col("p.equipments"), r"\s*;\s*"),
    lambda x: F.trim(x)
)

# 5) Construire le DataFrame silver (colonnes à plat)
silver_df = parsed.select(
    F.col("p.id").alias("id"),
    F.col("p.url").alias("url"),
    F.col("p.title").alias("title"),
    F.col("p.price_text").alias("price_text"),
    price_value.alias("price_value_mad"),
    F.col("p.category").alias("category"),
    F.col("p.breadcrumbs").alias("breadcrumbs"),
    F.col("p.description").alias("description"),
    F.col("p.seller_name").alias("seller_name"),
    F.col("p.seller_type").alias("seller_type"),
    F.col("p.published_date").alias("published_date_text"),
    image_urls_arr.alias("image_urls"),
    equipments_arr.alias("equipments"),
    attrs_map.alias("attributes_map"),
    # garder le timestamp d'ingestion s'il est présent dans ton raw_df
    *([F.col("ingest_ts")] if "ingest_ts" in raw_df.columns else [])
)

In [74]:
import pandas as pd
from IPython.display import display

# Keep things compact
pd.set_option("display.max_columns", 20)   # don't try to show hundreds
pd.set_option("display.max_colwidth", 80)  # clamp long cells to ~80 chars

# pick a small sample and flatten newlines so rows stay short
pdf = (silver_df.limit(10)
       .toPandas()
       .replace({r'[\r\n\t]+': ' '}, regex=True))

# simple, compact table with ellipsis in long cells
display(
    pdf.style
      .set_table_styles([
          {'selector': 'table', 'props': [('table-layout','fixed'), ('width','100%')]},
          {'selector': 'th, td', 'props': [
              ('max-width','280px'),
              ('white-space','nowrap'),
              ('overflow','hidden'),
              ('text-overflow','ellipsis')
          ]}
      ])
      .hide(axis='index')  # remove row numbers
)

id,url,title,price_text,price_value_mad,category,breadcrumbs,description,seller_name,seller_type,published_date_text,image_urls,equipments,attributes_map,ingest_ts
57153832,https://www.avito.ma/fr/sala_el_jadida/local/Magasin_à_vendre_57153832.htm,Magasin à vendre,None,nan,"Local, à vendre",Home Icon Accueil > Tout le Maroc > Salé > Sala El Jadida > Avito Immobilier > Local > Magasin à vendre,"Salam, magasin à vendre, si vous êtes intéressé, bienvenue, prix négociable.",Hicham Alaoui,Particulier,2025-11-03 01:07:16,"['https://content.avito.ma/classifieds/images/10141390515?t=images', 'https://content.avito.ma/classifieds/images/10141390516?t=images']","['Bon état', '13', '21+ ans', 'Sécurité']","{'Condition': 'Bon état', 'Surface totale': '13', 'Âge du bien': '21+ ans'}",2025-11-03 00:30:30.003000
57066031,https://www.avito.ma/fr/autre_secteur/local/Local_commercial_3_garages_57066031.htm,Local commercial 3 garages,15 000 DH,15000.000000,"Local, à vendre",Home Icon Accueil > Tout le Maroc > Casablanca > Autre secteur > Avito Immobilier > Local > Local commercial 3 garages,"Société by home vous propose des magasines sur * bouskoura Victoria superficies 66m2 commerce et 12m2 voiture * Dar bouaaza errahma 41m2, RDC 21m2 suppent et 50m2 cave. dans une résidence haut-standing. pour plus d'informations contactez nous.",By home,Particulier,2025-11-03 00:59:18,"['https://content.avito.ma/classifieds/images/10140466849?t=images', 'https://content.avito.ma/classifieds/images/10140466850?t=images', 'https://content.avito.ma/classifieds/images/10140466852?t=images', 'https://content.avito.ma/classifieds/images/10140466851?t=images']","['Neuf', 'Immédiate', '1', '66', 'Moins de 1 an', 'Chauffage', 'Climatisation', 'Parking', 'Sécurité']","{'Condition': 'Neuf', 'Disponibilité': 'Immédiate', 'Salle de bain': '1', 'Surface totale': '66', 'Âge du bien': 'Moins de 1 an'}",2025-11-03 00:30:30.003000
56708685,https://www.avito.ma/fr/alliance/appartements/À_VENDRE_Bel_appartement_lumineux_2_façades_56708685.htm,À VENDRE Bel appartement lumineux 2 façades,490 000 DH,490000.000000,"Appartements, à vendre",Home Icon Accueil > Tout le Maroc > Kénitra > alliance > Avito Immobilier > Ventes Immobilières > Appartements > À VENDRE Bel appartement lumineux 2 façades,"Bonjour, Je met en vente mon appartement presque neuf jamais habité situé au quartier Alliance Mehdia. Composition : 2 chambres, 1 salon, 2 salles de bain, 2 balcons. L’appartement est de deux façades opposées offrant une excellente aération et un ensoleillement optimal toute la journée. Orientation Rabat. Prix raisonnable et légèrement négociable. Les intermédiaires à s’abstenir svp.",Bouchra,Particulier,2025-11-03 00:58:20,"['https://content.avito.ma/classifieds/images/10137258041?t=images', 'https://content.avito.ma/classifieds/images/10137258042?t=images', 'https://content.avito.ma/classifieds/images/10137258040?t=images', 'https://content.avito.ma/classifieds/images/10137258043?t=images', 'https://content.avito.ma/classifieds/images/10137258038?t=images', 'https://content.avito.ma/classifieds/images/10137258045?t=images']","['2', '2', 'Neuf', 'Immédiate', '100', '1', 'Balcon']","{'Condition': 'Neuf', 'Disponibilité': 'Immédiate', 'Salle de bain': '2', 'Frais de syndic / mois': '100', 'Chambres': '2', 'Salons': '1'}",2025-11-03 00:30:30.003000
57133177,https://www.avito.ma/fr/autre_secteur/bureaux/Bureau_à_louer_Bendibane_57133177.htm,Bureau à louer Bendibane,5 500 DH,5500.000000,"Bureaux, à louer",Home Icon Accueil > Tout le Maroc > Tanger > Autre secteur > Avito Immobilier > Bureaux > Bureau à louer Bendibane,"Quatre bureaux à louer à Ben Diban1 au 3eme étage, 1 au 4eme étage, 2 au 5eme étage sur la rue principale. Du Rambouan du cinéma Tarek jusqu’à Ben D’iban emplacement stratégique sur la rue principale. Chaque bureau comprend 3 pièces, salon d’accueil, salle de bain, cuisine parmi les Activités autorisées : Médecin esthétique, dentiste, médecin généraliste, notaire. Conditions de loc

In [75]:
silver_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- title: string (nullable = true)
 |-- price_text: string (nullable = true)
 |-- price_value_mad: double (nullable = true)
 |-- category: string (nullable = true)
 |-- breadcrumbs: string (nullable = true)
 |-- description: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- seller_type: string (nullable = true)
 |-- published_date_text: string (nullable = true)
 |-- image_urls: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- equipments: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- attributes_map: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- ingest_ts: timestamp (nullable = true)



In [76]:
# Drop the 'price_text' column
silver_df = silver_df.drop('price_text')

# Rename 'price_value_mad' to 'price'
silver_df = silver_df.withColumnRenamed('price_value_mad', 'price')

# Replace NULL 'price' values with 0.0
silver_df = silver_df.fillna({'price': 0.0})

silver_df.select('price').show(10, truncate=False)

+-------+
|price  |
+-------+
|7500.0 |
|12500.0|
|6000.0 |
|0.0    |
|14000.0|
|6300.0 |
|7300.0 |
|0.0    |
|2300.0 |
|7500.0 |
+-------+
only showing top 10 rows



In [77]:
from pyspark.sql import functions as F

# Transform 'category' into 'offre' and 'category_type'
silver_df = (
    silver_df
    # Create 'offre' column for 'rent' or 'sale'
    .withColumn(
        "offre",
        F.when(F.col("category").contains("à louer"), "rent")
         .when(F.col("category").contains("à vendre"), "sale")
         .otherwise(None)
    )
    # Create 'category_type' column with only the property type (e.g., Maisons, Appartements)
    .withColumn(
        "type",
        F.when(
            F.col("category").contains("à louer") | F.col("category").contains("à vendre"),
            F.split(F.col("category"), ",")[0]
        ).otherwise(None)
    )
    # Drop the original 'category' column
    .drop("category")
)

# Check the result
silver_df.select("offre", "type").show(30, truncate=False)


+-----+------------------+
|offre|type              |
+-----+------------------+
|sale |Terrains et fermes|
|sale |Villas et Riads   |
|sale |Terrains et fermes|
|sale |Local             |
|sale |Appartements      |
|sale |Maisons           |
|rent |Bureaux           |
|rent |Appartements      |
|rent |Appartements      |
|rent |Appartements      |
|rent |Bureaux           |
|sale |Local             |
|sale |Local             |
|sale |Appartements      |
|rent |Appartements      |
|rent |Appartements      |
|rent |Appartements      |
|rent |Bureaux           |
|rent |Appartements      |
|rent |Bureaux           |
|rent |Appartements      |
|rent |Local             |
|rent |Appartements      |
|rent |Appartements      |
|rent |Appartements      |
|rent |Appartements      |
|rent |Local             |
|rent |Appartements      |
|rent |Appartements      |
|rent |Bureaux           |
+-----+------------------+
only showing top 30 rows



In [78]:
from pyspark.sql import functions as F

# Split 'breadcrumbs' based on '>'
split_breadcrumbs = F.split(F.col("breadcrumbs"), " > ")

# Create new columns for each segment
silver_df = (
    silver_df
    .withColumn("city", split_breadcrumbs.getItem(2))  # e.g., "Casablanca"
    .withColumn("neighborhood", split_breadcrumbs.getItem(3))  # e.g., "Maarif"
    .withColumn("site", split_breadcrumbs.getItem(4))  # e.g., "Avito Immobilier"
    .drop("breadcrumbs")  # Drop the original column if not needed
)

# Check the result
silver_df.select(
     "city", "neighborhood", "site", 
).show(30, truncate=False)


+----------+--------------+----------------+
|city      |neighborhood  |site            |
+----------+--------------+----------------+
|Casablanca|Lissasfa      |Avito Immobilier|
|Rabat     |Hay Riad      |Avito Immobilier|
|Marrakech |Targa         |Avito Immobilier|
|Casablanca|Val Fleuri    |Avito Immobilier|
|Casablanca|Bourgogne     |Avito Immobilier|
|Casablanca|Florida       |Avito Immobilier|
|Agadir    |Hay Dakhla    |Avito Immobilier|
|Mohammedia|Autre secteur |Avito Immobilier|
|Tit Mellil|Toute la ville|Avito Immobilier|
|Tanger    |Autre secteur |Avito Immobilier|
|Mohammedia|El Alia       |Avito Immobilier|
|Rabat     |Souissi       |Avito Immobilier|
|Meknès    |Hamria        |Avito Immobilier|
|Rabat     |Guich Oudaya  |Avito Immobilier|
|Casablanca|Anfa          |Avito Immobilier|
|Casablanca|C.I.L         |Avito Immobilier|
|Tanger    |De La Plage   |Avito Immobilier|
|Nouaceur  |Toute la ville|Avito Immobilier|
|Casablanca|Sidi Bernoussi|Avito Immobilier|
|Temara   

In [79]:
from pyspark.sql import functions as F

# Change all values in the 'site' column to 'avito'
silver_df = silver_df.withColumn('site', F.lit('avito'))

# Show the updated data
silver_df.select('site').show(10, truncate=False)

+-----+
|site |
+-----+
|avito|
|avito|
|avito|
|avito|
|avito|
|avito|
|avito|
|avito|
|avito|
|avito|
+-----+
only showing top 10 rows



In [89]:
# Group by 'seller_name' and 'seller_type', and count the occurrences
silver_df.groupBy('seller_type').count().orderBy('count', ascending=False).show(10, truncate=False)

+-----------+-----+
|seller_type|count|
+-----------+-----+
|Particulier|238  |
|Boutique   |1    |
|NULL       |1    |
+-----------+-----+



In [90]:
from pyspark.sql import functions as F

# Convert 'seller_type' to lowercase
silver_df = silver_df.withColumn("seller_type", F.lower(F.col("seller_type")))

# Show the distinct values of 'seller_type' with counts, including NULLs
silver_df.groupBy('seller_type').count().orderBy('count', ascending=False).show(30, truncate=False)

# Show rows where 'seller_type' is NULL
silver_df.filter(F.col('seller_type').isNull()).show(5, truncate=False)

+-----------+-----+
|seller_type|count|
+-----------+-----+
|particulier|238  |
|boutique   |1    |
|NULL       |1    |
+-----------+-----+

+--------+---------------------------------------------------------------------------------------------+-----+-----+-----------+-----------+-----------+----------+----------+-----------------------+-----+----+----+------------+-----+-----------------+-------+------+------------------+--------------+-----+-----------+-------------+----------------+--------+----------------------+---------+--------------+-------------+------+--------+--------------+
|id      |url                                                                                          |title|price|description|seller_name|seller_type|image_urls|equipments|ingest_ts              |offre|type|city|neighborhood|site |Surface habitable|Caution|Zoning|Type d'appartement|Surface totale|Étage|Âge du bien|Salle de bain|Nombre de pièces|Chambres|Frais de syndic / mois|Condition|Nombre d'étage|D

In [91]:
# Drop rows where 'seller_type' is NULL
silver_df = silver_df.filter(F.col('seller_type').isNotNull())

# Show the updated result to confirm the rows are dropped
silver_df.select('seller_type').distinct().show(10, truncate=False)

+-----------+
|seller_type|
+-----------+
|particulier|
|boutique   |
+-----------+



In [94]:
# Group by 'seller_name' and 'seller_type', and count the occurrences
silver_df.groupBy('seller_name').count().orderBy('count', ascending=False).show(5, truncate=False)

+----------------------------+-----+
|seller_name                 |count|
+----------------------------+-----+
|MedZaim immobilier          |10   |
|AGENCE IMMOBILIER MOSTAGIMMO|10   |
|SAKAN LIK GROUPE            |8    |
|SAFIN Immobilier            |6    |
|ABAMNY Immobilier           |6    |
+----------------------------+-----+
only showing top 5 rows



In [83]:
from pyspark.sql.functions import to_timestamp

# Convert 'published_date_text' to timestamp and rename it to 'published_date'
silver_df = silver_df.withColumn(
    "published_date", 
    to_timestamp("published_date_text", "yyyy-MM-dd HH:mm:ss")  # Adjust format if needed
).drop("published_date_text")  # Drop the original 'published_date_text' column

# Show the result
silver_df.select("id", "published_date").show(5, truncate=False)

+--------+-------------------+
|id      |published_date     |
+--------+-------------------+
|57020711|2025-11-02 11:17:06|
|57055336|2025-11-02 18:17:08|
|57140550|2025-11-02 13:17:10|
|56620071|2025-11-03 00:58:12|
|56920617|2025-11-03 00:59:14|
+--------+-------------------+
only showing top 5 rows



In [80]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import ArrayType, StringType
import re

# Define a UDF to clean the 'equipments' list, handling NoneType
def clean_equipments(equipments):
    # Handle NoneType case
    if equipments is None:
        return []
    
    # Filter out unwanted entries (numbers and time-related strings)
    cleaned = [item for item in equipments if not re.match(r'^\d+$', str(item)) and not re.match(r'.*(mois|an).*', str(item))]
    return cleaned

# Register the UDF
clean_equipments_udf = udf(clean_equipments, ArrayType(StringType()))

# Apply the UDF to clean 'equipments' directly (in place)
silver_df = silver_df.withColumn("equipments", clean_equipments_udf(col("equipments")))

# Show the cleaned 'equipments' column
silver_df.select("equipments").show(5, truncate=False)

+--------------------------------------------------------------+
|equipments                                                    |
+--------------------------------------------------------------+
|[Bon état, Sécurité]                                          |
|[Neuf, Immédiate, Chauffage, Climatisation, Parking, Sécurité]|
|[Neuf, Immédiate, Balcon]                                     |
|[Ascenseur]                                                   |
|[]                                                            |
+--------------------------------------------------------------+
only showing top 5 rows



In [81]:
from pyspark.sql import functions as F

# Extract the keys from 'attributes_map'
keys = silver_df.select(F.explode(F.map_keys(F.col("attributes_map"))).alias("attribute")).distinct().rdd.flatMap(lambda x: x).collect()

# For each key, create a new column with its corresponding value from 'attributes_map'
for key in keys:
    silver_df = silver_df.withColumn(
        key, 
        F.when(F.col("attributes_map").getItem(key).isNotNull(), 
               F.col("attributes_map").getItem(key)).otherwise(None)
    )

# Drop the 'attributes_map' column after extracting the keys and values
silver_df = silver_df.drop("attributes_map")

# Show the result with the new columns and drop the 'attributes_map' column
silver_df.select("id", *keys).show(5, truncate=False)

+--------+-----------------+-------+------+------------------+--------------+-----+-----------+-------------+----------------+--------+----------------------+---------+--------------+-------------+------+----------+
|id      |Surface habitable|Caution|Zoning|Type d'appartement|Surface totale|Étage|Âge du bien|Salle de bain|Nombre de pièces|Chambres|Frais de syndic / mois|Condition|Nombre d'étage|Disponibilité|Salons|Standing  |
+--------+-----------------+-------+------+------------------+--------------+-----+-----------+-------------+----------------+--------+----------------------+---------+--------------+-------------+------+----------+
|56941776|NULL             |NULL   |NULL  |NULL              |290           |NULL |NULL       |4            |NULL            |4       |NULL                  |Bon état |4             |Immédiate    |NULL  |NULL      |
|57106819|NULL             |NULL   |NULL  |NULL              |NULL          |NULL |NULL       |NULL         |NULL            |NULL    |N

In [95]:
import pandas as pd
from IPython.display import display

# Keep things compact
pd.set_option("display.max_columns", 20)   # don't try to show hundreds
pd.set_option("display.max_colwidth", 80)  # clamp long cells to ~80 chars

# pick a small sample and flatten newlines so rows stay short
pdf = (silver_df.limit(10)
       .toPandas()
       .replace({r'[\r\n\t]+': ' '}, regex=True))

# simple, compact table with ellipsis in long cells
display(
    pdf.style
      .set_table_styles([
          {'selector': 'table', 'props': [('table-layout','fixed'), ('width','100%')]},
          {'selector': 'th, td', 'props': [
              ('max-width','280px'),
              ('white-space','nowrap'),
              ('overflow','hidden'),
              ('text-overflow','ellipsis')
          ]}
      ])
      .hide(axis='index')  # remove row numbers
)

id,url,title,price,description,seller_name,seller_type,image_urls,equipments,ingest_ts,offre,type,city,neighborhood,site,Surface habitable,Caution,Zoning,Type d'appartement,Surface totale,Étage,Âge du bien,Salle de bain,Nombre de pièces,Chambres,Frais de syndic / mois,Condition,Nombre d'étage,Disponibilité,Salons,Standing,published_date
57020711,https://www.avito.ma/fr/lissasfa/bureaux/Bureau_à_louer_d_une_superficie_141_m²_57020711.htm,Bureau à louer d'une superficie 141 m²,12690.000000,"FBC met à votre disposition un plateau bureau 311 à louer de 141 m², situé dans un espace professionnel, sécurisé, calme et propre. Situé au 3ème étage, bien équipé : climatisation, 2 ascenseurs, cloisons, chauffage, parking et internet haut débit (fibre optique), kitchenette, accès direct à deux autoroutes, celle de rond-point Azbane et la Rocade sud-ouest. Pour plus d'information, n'hésitez pas à nous contacter.",FACILITIES BUSINESS CENTER,particulier,"['https://content.avito.ma/classifieds/images/10139961083?t=images', 'https://content.avito.ma/classifieds/images/10139961084?t=images', 'https://content.avito.ma/classifieds/images/10139961070?t=images', 'https://content.avito.ma/classifieds/images/10139961074?t=images', 'https://content.avito.ma/classifieds/images/10139961073?t=images', 'https://content.avito.ma/classifieds/images/10139961069?t=images', 'https://content.avito.ma/classifieds/images/10139961068?t=images', 'https://content.avito.ma/classifieds/images/10139961072?t=images', 'https://content.avito.ma/classifieds/images/10139961076?t=images', 'https://content.avito.ma/classifieds/images/10139961077?t=images', 'https://content.avito.ma/classifieds/images/10139961078?t=images', 'https://content.avito.ma/classifieds/images/10139961082?t=images']","['Ascenseur', 'Chauffage', 'Climatisation', 'Câblage téléphonique', 'Parking', 'Sécurité']",2025-11-03 00:17:15.003000,rent,Bureaux,Casablanca,Lissasfa,avito,None,None,None,None,141,3,None,0,3,None,None,None,None,None,None,None,2025-11-02 11:17:06
57055336,https://www.avito.ma/fr/hay_riad/appartements/Appartement_en_location_à_Hay_Riad_Rabat_57055336.htm,Appartement en location à Hay Riad Rabat,0.000000,"Bel appartement de 195 m² en location au 1 er étage situé à Hay Riad Rabat Composé de deux salons avec cheminée séjour cuisine salle d'inviter deux chambres avec salle de bain suite parental avec salle de bain et drissing prix de location 16000dhs / mois pour plus d'informations veuillez contacter nous ,",Immosaadaoui,particulier,"['https://content.avito.ma/classifieds/images/10140311388?t=images', 'https://content.avito.ma/classifieds/images/10140311398?t=images', 'https://content.avito.ma/classifieds/images/10140311399?t=images', 'https://content.avito.ma/classifieds/images/10140311404?t=images', 'https://content.avito.ma/classifieds/images/10140311389?t=images', 'https://content.avito.ma/classifieds/images/10140311393?t=images', 'https://content.avito.ma/classifieds/images/10140311387?t=images']",[],2025-11-03 00:17:15.003000,rent,Appartements,Rabat,Hay Riad,avito,195,None,None,None,None,1,None,None,None,4,None,None,None,None,None,None,2025-11-02 18:17:08
57140550,https://www.avito.ma/fr/targa/bureaux/Bureaux_privés_Domiciliation_à_Targa__Marrakech_57140550.htm,"Bureaux privés-Domiciliation à Targa, Marrakech",1200.000000,🏢 Bureaux privés & domiciliation à Marrakech Besoin d’un espace professionnel clé en main à Marrakech ? Excellentia Business Center vous propose : ✅ Bureaux privés équipés et climatisés – prêts à l’emploi ✅ Internet fibre optique haut débit inclus ✅ Service de domiciliation 100 % en ligne – obtenez votre adresse professionnelle sans vous déplacer ✅ Réception et gestion de votre courrier Offrez à votre entreprise une image professionnelle et un environnement de travail moderne au cœur de Marrakech. 📞 Contactez-nous dès aujourd’hui et profitez d’une solution complète pour votre bureau et domiciliation.,Excellentia Business Center,particulier,"['https://content.avito.ma/c

In [85]:
silver_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- title: string (nullable = true)
 |-- price: double (nullable = false)
 |-- description: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- seller_type: string (nullable = true)
 |-- image_urls: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- equipments: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ingest_ts: timestamp (nullable = true)
 |-- offre: string (nullable = true)
 |-- type: string (nullable = true)
 |-- city: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- site: string (nullable = false)
 |-- Surface habitable: string (nullable = true)
 |-- Caution: string (nullable = true)
 |-- Zoning: string (nullable = true)
 |-- Type d'appartement: string (nullable = true)
 |-- Surface totale: string (nullable = true)
 |-- Étage: string (nullable = true)
 |-- Âge du bien: string (nullable = true)
 |-- Sa